In [1]:
from utils import compute_f1, load_data
from text_train.imports import *
from text_train.data import TextDS, text_to_device
from arcface import ArcMarginProduct
from text_train.model import EMBBert
from text_train.train import *
np.random.seed(1337)
device = torch.device('cuda')

In [ ]:
df, train_df, val_df, train_labels, val_labels = load_data(train_perc=0.3)